# **F-Algebra and F-Coalbgera**


In [1]:
using Pkg
Pkg.activate(".")
using MLStyle

  Activating project at `~/Documents/GitHub/Julia_Tutorials/CategoryTheory`


In [2]:
struct NilF end
struct ConsF{c,a}
    _1::c
    _2::a
end
ListF{c,a} = Union{NilF, ConsF{c,a}}
fmap(f::Function, x::ConsF) = ConsF(x._1,f(x._2))
fmap(f::Function, x::NilF)  = NilF()


# data List c = Nil | Cons c (List c) ----- In Haskell
struct Nil end
struct Cons{c}
    _1::c
    _2::Union{Nil, Cons{c}}
end

List{T} = Union{Nil, Cons{T}};

In [3]:
fix(::NilF)  = Nil()
fix(x::ConsF)= Cons(x._1,x._2)
fix(ConsF(10, Nil()))

Cons{Int64}(10, Nil())

Note that `ListF{I,T}` is a functor when we fix the first type. Fix `I` to be `Int`. Now we have
`ListF{Int,T}`, which we use to create the category of algebras over.
The (`List{Int}`,`fix`) algebra is the initial object in the `ListF{Int,_}`-Algebra category.
Let's define another F-Algebra for the functor `ListF{Int,_}`. For that, we need a *carrier* type `T`
and an evaluation function `eval(x::ListF{Int,T})::T`.

Take `T` to be `Int`. We can define the evaluation as the following:

In [4]:
evalSum(::NilF)   = 0
evalSum(x::ConsF{Int,Int}) = x._1 + x._2;

In [5]:
cataSum(::Nil)   = evalSum(NilF())
cataSum(x::Cons) = evalSum(ConsF(x._1,(cataSum(x._2))))
cataSum(Cons(10, Cons(20,Nil())))

30

Note that (`Int`, `evalSum`) is a `ListF{Int,_}`-Algebra. With the same carrier (`Int`) we can
define a new algebra by changing `evalSum` for `evalProd`.

Hence, we get a new catamorphism `cataProd`.

In [6]:
evalProd(::NilF)   = 1
evalProd(x::ConsF{Int,Int}) = x._1 * x._2;

cataProd(::Nil)   = evalProd(NilF())
cataProd(x::Cons) = evalProd(ConsF(x._1,(cataProd(x._2))))
cataProd(Cons(10, Cons(20,Nil())))

200

### Catamorphism

We can perceive that there is a general construction. Which is defined as

```Haskell
cata alg = alg . fmap (cata alg) . unFix
```

A theorem by Lambek states that for the initial F-Algebra, which is `(List{c}, fix)` in our case,
there is an inverse function to `fix`, which we call `unfix`. While
`fix(x::ListF{c, List{c}})::List{c}`, the inverse take a vaue of `List{c}` and returns
a value of type `ListF{c,List{c}}`. Let's implement this.

In [7]:
unfix(::Nil) = NilF()
unfix(x::Cons)= ConsF(x._1,x._2)
unfix(Cons(10, Nil()))

ConsF{Int64, Nil}(10, Nil())

We can now implement the generic catamorphism. Note that the idea is to
do a single implementation `cata` that takes an algebra (`eval`), instead
of implementing every single `cataSum`,  `cataProd` and so on.

In [8]:
x = Cons(20,Cons(10,Nil()))

cata(alg::Function, x::List) = alg(fmap(y -> cata(alg, y), unfix(x)))
@show cata(evalSum,Nil())
@show cata(evalSum,Cons(10,Nil()))
@show cata(evalSum,Cons(10,Cons(10,Nil())));

@show cata(evalProd,Nil())
@show cata(evalProd,Cons(10,Nil()))
@show cata(evalProd,Cons(10,Cons(10,Nil())));

cata(evalSum, Nil()) = 0
cata(evalSum, Cons(10, Nil())) = 10
cata(evalSum, Cons(10, Cons(10, Nil()))) = 20
cata(evalProd, Nil()) = 1
cata(evalProd, Cons(10, Nil())) = 10
cata(evalProd, Cons(10, Cons(10, Nil()))) = 100


In [9]:
# cata(evalSum, Nil())
fmap(y->cata(evalSum,y),unfix(Cons(10,Nil())))

ConsF{Int64, Int64}(10, 0)

**Example 2 - Natural Numbers Functor** 

In [10]:
@data NatF begin
    ZeroF()
    SuccF(_1)
end
fmap(f::Function, x::ZeroF)  = ZeroF()
fmap(f::Function, x::SuccF) = SuccF(f(x._1))

@data Nat begin
    Zero()
    Succ(Nat)
end
unfix(::Zero) = ZeroF()
unfix(x::Succ)= SuccF(x._1)
unfix(Succ(Succ(Zero())))

fib(::ZeroF) = (1,1)
fib(x::SuccF) = (x._1[2], x._1[1] + x._1[1])

@show fib(SuccF((10,10)))
@show fib(ZeroF());

cata(alg::Function, x::Nat) = alg(fmap(y -> cata(alg, y), unfix(x)))

@show cata(fib, Succ(Succ(Zero())));

fib(SuccF((10, 10))) = (10, 20)
fib(ZeroF()) = (1, 1)
cata(fib, Succ(Succ(Zero()))) = (2, 2)


## Vector to List

In Julia, we already have the Vector type to represent a List. We can define a function that turns a Vector into our List type.

In [11]:
function vectolist(x::Vector)
    if x == []
        return Nil()
    end
    (n,ns) = Iterators.peel(x)
    ns = collect(ns)
    return Cons(n,vectolist(ns))
end

listtovec(x::Nil) = []
listtovec(x::Cons{T}) where T = convert(Vector{T},vcat(x._1,listtovec(x._2)))
listtovec(vectolist([1,2,3]))

3-element Vector{Int64}:
 1
 2
 3

## Co-Algebras and Anamorphism
The co-algebra is the dual of an F-Algebra:
$$
a \to F a
$$


Instead of our `List` , let us use the `Vector` from Julia.

In [12]:
# Make `Vector` into a functor by defining the fmap
fmap(f,x) = map(f,x)

fmap (generic function with 5 methods)

In [13]:
@data TreeF{a} begin
    LeafF()
    NodeF(Int,::a, ::a)
end
fmap(f::Function, x::LeafF) = x
fmap(f::Function, x::NodeF) = NodeF(x._1,f(x._2),f(x._3))
NodeF(x::Int,y::TreeF{T},z::TreeF{T}) where T = NodeF{TreeF{T}}(x,y,z) 

@data Tree begin
    Leaf()
    Node(Int,::Tree, ::Tree)
end

fix(x::LeafF)  = Leaf()
fix(x::NodeF)= Node(x._1,x._2,x._3)

NodeF(x::Int,y::Tree, z::Tree) = NodeF{Tree}(x::Int,y,z)
fix(NodeF(10,fix(LeafF{Int}()),fix(LeafF{Int}())))

Node(10, Leaf(), Leaf())

In [14]:
x = [1,2,3,4,-12,0,5]
function partition(f, x::Vector{T}) where T
    index = map(f, x)
    x[index],x[.!(index)]
end
partition(x-> x ≤ 3,x)

([1, 2, 3, -12, 0], [4, 5])

In [15]:
function split(ns::Vector{Int})::TreeF{Vector{Int}}
    if isempty(ns)
        return LeafF{Vector{Int}}()
    end
    
    # Extract head and tail [1,2,3], after peel we get n = 1, ns = [2,3]
    # ns comes out as an iterator, hence we use `collect`.
    (n,ns) = Iterators.peel(ns)
    ns = collect(ns)
    
    (left, right) = partition(x -> x <= n, ns)
    return NodeF(n, left, right)
end

split (generic function with 1 method)

In [16]:
split(Int[])

LeafF{Vector{Int64}}()

In [17]:
split([1])

NodeF{Vector{Int64}}(1, Int64[], Int64[])

In [18]:
split([1,2,0,1,3])

NodeF{Vector{Int64}}(1, [0, 1], [2, 3])

```
ana :: Functor f => Coalgebra f a -> a -> Fix f
ana coa = In . fmap (ana coa) . coa
``` 

In [ ]:
fmap(f::Function) = x->fmap(f, x)

In [23]:
# Let coalg: A -> [A] , i.e. List coalgebras
ana(coalg::Function, x) = fix(fmap(y -> ana(coalg, y), coalg(x)))

ana (generic function with 1 method)

In [24]:
ana(split,Int[])

Leaf()

In [25]:
ana(split,[1])

Node(1, Leaf(), Leaf())

In [27]:
ana(split,[1,2,3,4,5])

Node(1, Leaf(), Node(2, Leaf(), Node(3, Leaf(), Node(4, Leaf(), Node(5, Leaf(), Leaf())))))